In [3]:
import xarray as xr
import os
import requests
from shapely.geometry import mapping, shape
from shapely.prepared import prep
from shapely.geometry import Point

In [4]:
ds = xr.open_dataset("/content/dgm_ensmean_rcp26_ssp1soc_lts_2010_2039.nc4")
df = ds.to_dataframe().reset_index()


data = requests.get("https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson").json()

countries = {}
for feature in data["features"]:
    geom = feature["geometry"]
    country = feature["properties"]["ADMIN"]
    countries[country] = prep(shape(geom))


def get_country(row):
    lat = row['lat']
    lon = row['lon']
    point = Point(lon, lat)
    for country, geom in countries.items():
        if geom.contains(point):
            return country

    return "unknown"

def aggregate_country_lts(df):
    country_df = df[df['country'] != 'unknown']
    aggregated_df = country_df.groupby('country')['lts'].aggregate(['min','max', 'mean'])
    return aggregated_df



In [ ]:
df['country'] = df.apply (lambda row: get_country(row), axis=1)
aggregated_df = aggregate_country_lts(df)

In [ ]:
df.to_csv("country_LTS.csv")
aggregated_df.to_csv("aggregated_country_LTS.csv")